# 🎯 PhD Research Dashboard - ULTRA-CUSTOMIZABLE INTERACTIVE NOTEBOOK

## RIS Probe-Based ML Research Interface

This comprehensive dashboard provides **COMPLETE CONTROL** over all experimental parameters through an intuitive widget-based interface.

### 🌟 Key Features:

- **5 Organized Tabs**: System & Physics, Model Architecture, Training, Evaluation, Visualization
- **Dynamic Interactions**: Smart widget dependencies and auto-validation
- **Multi-Model Comparison**: Run and compare multiple architectures
- **Statistical Analysis**: Multi-seed runs with confidence intervals
- **Publication-Ready Plots**: 20+ plot types with customization
- **Export System**: Organized output structure with config tracking
- **Reproducibility**: Save/load configs, track all parameters

### 📋 Quick Start:

1. Run all cells sequentially (Cell → Run All)
2. Configure parameters in the widget tabs
3. Click "🚀 RUN EXPERIMENT"
4. View real-time progress and results

### 🎓 Ideal for:

- PhD research exploration
- Hyperparameter sweeps
- Model architecture comparison
- Publication-quality results

---

**Author**: Mohammad Bagher  
**Repository**: [Probe_Based_ML_Session5](https://github.com/mohbagher/Probe_Based_ML_Session5)  
**License**: MIT

In [ ]:
# ============================================================================
# CELL 2: IMPORTS & SETUP
# ============================================================================

import sys
import os
import json
import yaml
import time
import pickle
from datetime import datetime
from pathlib import Path
from typing import Dict, List, Optional, Any, Tuple
import warnings

# Scientific computing
import numpy as np
import pandas as pd
from scipy import stats

# Deep learning
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.gridspec import GridSpec

# Jupyter widgets
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output, Markdown
from ipywidgets import (
    IntSlider, FloatSlider, FloatLogSlider, Dropdown, Checkbox,
    IntText, Text, Textarea, Button, HBox, VBox, Tab,
    Label, Layout, SelectMultiple, Output, HTML as HTMLWidget
)

# Progress bars
from tqdm.auto import tqdm

# Local imports
from config import Config, SystemConfig, DataConfig, ModelConfig, TrainingConfig, EvalConfig, get_config
from model import LimitedProbingMLP, create_model, count_parameters
from data_generation import create_dataloaders
from training import train
from evaluation import evaluate_model, compute_baselines
from experiments.probe_generators import get_probe_bank, ProbeBank
from model_registry import MODEL_REGISTRY, list_models, get_model_architecture, register_model
from plot_registry import PLOT_REGISTRY, list_plots, get_plot_function

# Configuration
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('viridis')

# Check GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"✓ Using device: {device}")
if torch.cuda.is_available():
    print(f"  GPU: {torch.cuda.get_device_name(0)}")
    print(f"  Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

print(f"✓ PyTorch version: {torch.__version__}")
print(f"✓ NumPy version: {np.__version__}")
print(f"✓ IPyWidgets version: {widgets.__version__}")
print("\n✓ All imports successful!")

In [ ]:
# ============================================================================
# CELL 3: WIDGET-BASED CONTROL PANEL (Organized in Tabs)
# ============================================================================

# Global widget storage
widgets_dict = {}

# ====================
# TAB 1: SYSTEM & PHYSICS
# ====================

# System Parameters
widgets_dict['N'] = IntSlider(
    value=32, min=8, max=256, step=8,
    description='N (RIS Elements):',
    style={'description_width': '150px'},
    layout=Layout(width='500px'),
    continuous_update=False
)

widgets_dict['K'] = IntSlider(
    value=64, min=16, max=512, step=16,
    description='K (Codebook Size):',
    style={'description_width': '150px'},
    layout=Layout(width='500px'),
    continuous_update=False
)

widgets_dict['M'] = IntSlider(
    value=8, min=1, max=64,
    description='M (Sensing Budget):',
    style={'description_width': '150px'},
    layout=Layout(width='500px'),
    continuous_update=False
)

widgets_dict['P_tx'] = FloatSlider(
    value=1.0, min=0.1, max=10.0, step=0.1,
    description='P_tx (Power):',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['sigma_h_sq'] = FloatSlider(
    value=1.0, min=0.1, max=5.0, step=0.1,
    description='σ²_h (BS-RIS):',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['sigma_g_sq'] = FloatSlider(
    value=1.0, min=0.1, max=5.0, step=0.1,
    description='σ²_g (RIS-UE):',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

# Phase Configuration
widgets_dict['phase_mode'] = Dropdown(
    options=['Continuous', 'Discrete'],
    value='Continuous',
    description='Phase Mode:',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['phase_bits'] = IntSlider(
    value=2, min=1, max=8,
    description='Phase Bits:',
    style={'description_width': '150px'},
    layout=Layout(width='500px'),
    disabled=True
)

# Channel Model
widgets_dict['channel_type'] = Dropdown(
    options=['Rayleigh', 'Rician', 'Sparse', 'Correlated', 'Time-Varying'],
    value='Rayleigh',
    description='Channel Type:',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['rician_k'] = FloatSlider(
    value=5.0, min=0.1, max=20.0, step=0.1,
    description='Rician K-factor:',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['num_paths'] = IntSlider(
    value=3, min=1, max=10,
    description='Num Paths:',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['correlation_distance'] = FloatSlider(
    value=0.5, min=0.0, max=1.0, step=0.01,
    description='Correlation Dist:',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['doppler_freq'] = FloatSlider(
    value=0.01, min=0.0, max=0.1, step=0.001,
    description='Doppler Freq:',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

# Probe Methods
widgets_dict['probe_methods'] = SelectMultiple(
    options=['continuous', 'binary', '2bit', 'hadamard', 'sobol', 'halton'],
    value=['sobol', 'hadamard'],
    description='Probe Methods:',
    style={'description_width': '150px'},
    layout=Layout(width='500px', height='120px')
)

# Build Tab 1 layout
tab1_system = VBox([
    Label('System Parameters', layout=Layout(font_weight='bold')),
    widgets_dict['N'],
    widgets_dict['K'],
    widgets_dict['M'],
    widgets_dict['P_tx'],
    widgets_dict['sigma_h_sq'],
    widgets_dict['sigma_g_sq'],
])

tab1_phase = VBox([
    Label('Phase Configuration', layout=Layout(font_weight='bold')),
    widgets_dict['phase_mode'],
    widgets_dict['phase_bits'],
])

tab1_channel = VBox([
    Label('Channel Model', layout=Layout(font_weight='bold')),
    widgets_dict['channel_type'],
    widgets_dict['rician_k'],
    widgets_dict['num_paths'],
    widgets_dict['correlation_distance'],
    widgets_dict['doppler_freq'],
])

tab1_probe = VBox([
    Label('Probe Methods (Select Multiple)', layout=Layout(font_weight='bold')),
    widgets_dict['probe_methods'],
])

tab1 = VBox([
    HTML("<h3>🎛️ System & Physics</h3>"),
    HBox([tab1_system, tab1_phase]),
    HBox([tab1_channel, tab1_probe]),
], layout=Layout(padding='10px'))

# ====================
# TAB 2: MODEL ARCHITECTURE
# ====================

widgets_dict['model_type'] = Dropdown(
    options=['MLP', 'Attention', 'CNN', 'LSTM', 'GRU', 'ResNet-MLP', 'Transformer', 'Hybrid-CNN-LSTM'],
    value='MLP',
    description='Model Type:',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

# MLP Configuration
widgets_dict['mlp_num_layers'] = IntSlider(
    value=3, min=1, max=10,
    description='Num Layers:',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['mlp_layer_1'] = IntSlider(
    value=512, min=16, max=2048, step=16,
    description='Layer 1 Size:',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['mlp_layer_2'] = IntSlider(
    value=256, min=16, max=2048, step=16,
    description='Layer 2 Size:',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['mlp_layer_3'] = IntSlider(
    value=128, min=16, max=2048, step=16,
    description='Layer 3 Size:',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['use_skip_connections'] = Checkbox(
    value=False,
    description='Use Skip Connections',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

# CNN Configuration
widgets_dict['cnn_num_layers'] = IntSlider(
    value=3, min=1, max=5,
    description='Conv Layers:',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['pooling_type'] = Dropdown(
    options=['Max', 'Average'],
    value='Max',
    description='Pooling Type:',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['cnn_fc_hidden'] = Text(
    value='256,128',
    description='FC Hidden Sizes:',
    placeholder='comma-separated',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

# LSTM/GRU Configuration
widgets_dict['rnn_hidden_size'] = IntSlider(
    value=128, min=32, max=512, step=32,
    description='Hidden Size:',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['rnn_num_layers'] = IntSlider(
    value=2, min=1, max=5,
    description='RNN Layers:',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['bidirectional'] = Checkbox(
    value=True,
    description='Bidirectional',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['rnn_fc_hidden'] = Text(
    value='128,64',
    description='FC Hidden Sizes:',
    placeholder='comma-separated',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

# Attention/Transformer Configuration
widgets_dict['d_model'] = IntSlider(
    value=128, min=64, max=512, step=64,
    description='d_model:',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['num_attention_heads'] = Dropdown(
    options=[1, 2, 4, 8, 16],
    value=8,
    description='Attention Heads:',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['num_transformer_layers'] = IntSlider(
    value=3, min=1, max=12,
    description='Transformer Layers:',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['feedforward_dim'] = IntSlider(
    value=512, min=128, max=2048, step=128,
    description='Feedforward Dim:',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['use_positional_encoding'] = Checkbox(
    value=True,
    description='Positional Encoding',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

# Regularization
widgets_dict['dropout'] = FloatSlider(
    value=0.1, min=0.0, max=0.9, step=0.05,
    description='Dropout:',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['batch_normalization'] = Checkbox(
    value=True,
    description='Batch Normalization',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['layer_normalization'] = Checkbox(
    value=False,
    description='Layer Normalization',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['weight_init'] = Dropdown(
    options=['Xavier', 'Kaiming', 'Normal', 'Orthogonal'],
    value='Xavier',
    description='Weight Init:',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

# Build Tab 2 layout (dynamically shown/hidden based on model type)
mlp_config_box = VBox([
    Label('MLP Configuration', layout=Layout(font_weight='bold')),
    widgets_dict['mlp_num_layers'],
    widgets_dict['mlp_layer_1'],
    widgets_dict['mlp_layer_2'],
    widgets_dict['mlp_layer_3'],
    widgets_dict['use_skip_connections'],
])

cnn_config_box = VBox([
    Label('CNN Configuration', layout=Layout(font_weight='bold')),
    widgets_dict['cnn_num_layers'],
    widgets_dict['pooling_type'],
    widgets_dict['cnn_fc_hidden'],
])

rnn_config_box = VBox([
    Label('LSTM/GRU Configuration', layout=Layout(font_weight='bold')),
    widgets_dict['rnn_hidden_size'],
    widgets_dict['rnn_num_layers'],
    widgets_dict['bidirectional'],
    widgets_dict['rnn_fc_hidden'],
])

attention_config_box = VBox([
    Label('Attention/Transformer Configuration', layout=Layout(font_weight='bold')),
    widgets_dict['d_model'],
    widgets_dict['num_attention_heads'],
    widgets_dict['num_transformer_layers'],
    widgets_dict['feedforward_dim'],
    widgets_dict['use_positional_encoding'],
])

regularization_box = VBox([
    Label('Regularization', layout=Layout(font_weight='bold')),
    widgets_dict['dropout'],
    widgets_dict['batch_normalization'],
    widgets_dict['layer_normalization'],
    widgets_dict['weight_init'],
])

tab2 = VBox([
    HTML("<h3>🧠 Model Architecture</h3>"),
    widgets_dict['model_type'],
    mlp_config_box,
    cnn_config_box,
    rnn_config_box,
    attention_config_box,
    regularization_box,
], layout=Layout(padding='10px'))

print("✓ Tab 1 & 2 widgets created")


In [ ]:
# ====================
# TAB 3: TRAINING CONFIGURATION
# ====================

# Data Parameters
widgets_dict['n_train'] = IntText(
    value=50000,
    description='Train Samples:',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['n_val'] = IntText(
    value=5000,
    description='Val Samples:',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['n_test'] = IntText(
    value=5000,
    description='Test Samples:',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['seed'] = IntText(
    value=42,
    description='Random Seed:',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['normalize_input'] = Checkbox(
    value=True,
    description='Normalize Input',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['normalization_type'] = Dropdown(
    options=['mean', 'std', 'log', 'minmax'],
    value='mean',
    description='Normalization:',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

# Training Settings
widgets_dict['epochs'] = IntSlider(
    value=50, min=1, max=200,
    description='Epochs:',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['batch_size'] = Dropdown(
    options=[16, 32, 64, 128, 256, 512],
    value=128,
    description='Batch Size:',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['early_stop_patience'] = IntSlider(
    value=10, min=1, max=50,
    description='Early Stop:',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['validation_interval'] = IntSlider(
    value=1, min=1, max=10,
    description='Val Interval:',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

# Optimizer
widgets_dict['optimizer'] = Dropdown(
    options=['Adam', 'AdamW', 'SGD', 'RMSprop', 'AdaGrad', 'Adadelta', 'Adamax'],
    value='Adam',
    description='Optimizer:',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['learning_rate'] = FloatLogSlider(
    value=1e-3, min=-5, max=-1, step=0.1,
    description='Learning Rate:',
    style={'description_width': '150px'},
    layout=Layout(width='500px'),
    readout_format='.2e'
)

widgets_dict['weight_decay'] = FloatLogSlider(
    value=1e-4, min=-10, max=-1, step=0.1,
    description='Weight Decay:',
    style={'description_width': '150px'},
    layout=Layout(width='500px'),
    readout_format='.2e'
)

# SGD-specific
widgets_dict['momentum'] = FloatSlider(
    value=0.9, min=0.0, max=0.99, step=0.01,
    description='Momentum:',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['nesterov'] = Checkbox(
    value=False,
    description='Nesterov',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

# Adam-specific
widgets_dict['beta1'] = FloatSlider(
    value=0.9, min=0.5, max=0.999, step=0.001,
    description='Beta1:',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['beta2'] = FloatSlider(
    value=0.999, min=0.9, max=0.9999, step=0.0001,
    description='Beta2:',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['epsilon'] = FloatLogSlider(
    value=1e-8, min=-10, max=-6, step=0.1,
    description='Epsilon:',
    style={'description_width': '150px'},
    layout=Layout(width='500px'),
    readout_format='.2e'
)

# Learning Rate Scheduler
widgets_dict['scheduler'] = Dropdown(
    options=['None', 'StepLR', 'MultiStepLR', 'ExponentialLR', 'CosineAnnealing', 
             'CosineWarmup', 'ReduceLROnPlateau', 'OneCycleLR'],
    value='None',
    description='LR Scheduler:',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['step_size'] = IntSlider(
    value=10, min=1, max=50,
    description='Step Size:',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['gamma'] = FloatSlider(
    value=0.1, min=0.1, max=0.9, step=0.05,
    description='Gamma:',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['milestones'] = Text(
    value='30,60,90',
    description='Milestones:',
    placeholder='comma-separated',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['t_max'] = IntSlider(
    value=50, min=10, max=200,
    description='T_max:',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['eta_min'] = FloatLogSlider(
    value=1e-8, min=-8, max=-3, step=0.1,
    description='Eta Min:',
    style={'description_width': '150px'},
    layout=Layout(width='500px'),
    readout_format='.2e'
)

widgets_dict['warmup_epochs'] = IntSlider(
    value=5, min=0, max=20,
    description='Warmup Epochs:',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['plateau_mode'] = Dropdown(
    options=['min', 'max'],
    value='max',
    description='Mode:',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['plateau_factor'] = FloatSlider(
    value=0.5, min=0.1, max=0.9, step=0.05,
    description='Factor:',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['plateau_patience'] = IntSlider(
    value=5, min=1, max=20,
    description='Patience:',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['max_lr'] = FloatLogSlider(
    value=1e-2, min=-4, max=0, step=0.1,
    description='Max LR:',
    style={'description_width': '150px'},
    layout=Layout(width='500px'),
    readout_format='.2e'
)

# Loss Function
widgets_dict['loss_type'] = Dropdown(
    options=['CrossEntropy', 'Focal', 'LabelSmoothing'],
    value='CrossEntropy',
    description='Loss Function:',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['focal_alpha'] = FloatSlider(
    value=1.0, min=0.1, max=5.0, step=0.1,
    description='Focal Alpha:',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['focal_gamma'] = FloatSlider(
    value=2.0, min=0.0, max=5.0, step=0.5,
    description='Focal Gamma:',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['label_smoothing'] = FloatSlider(
    value=0.1, min=0.0, max=0.5, step=0.05,
    description='Smoothing:',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

# Advanced Regularization
widgets_dict['use_mixup'] = Checkbox(
    value=False,
    description='Use Mixup',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['mixup_alpha'] = FloatSlider(
    value=1.0, min=0.1, max=2.0, step=0.1,
    description='Mixup Alpha:',
    style={'description_width': '150px'},
    layout=Layout(width='500px'),
    disabled=True
)

widgets_dict['gradient_clipping'] = FloatSlider(
    value=0.0, min=0.0, max=10.0, step=0.5,
    description='Grad Clipping:',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['l1_regularization'] = FloatLogSlider(
    value=0.0, min=-10, max=-2, step=0.1,
    description='L1 Reg:',
    style={'description_width': '150px'},
    layout=Layout(width='500px'),
    readout_format='.2e'
)

# Build Tab 3 layout
data_box = VBox([
    Label('Data Parameters', layout=Layout(font_weight='bold')),
    widgets_dict['n_train'],
    widgets_dict['n_val'],
    widgets_dict['n_test'],
    widgets_dict['seed'],
    widgets_dict['normalize_input'],
    widgets_dict['normalization_type'],
])

training_box = VBox([
    Label('Training Settings', layout=Layout(font_weight='bold')),
    widgets_dict['epochs'],
    widgets_dict['batch_size'],
    widgets_dict['early_stop_patience'],
    widgets_dict['validation_interval'],
])

optimizer_box = VBox([
    Label('Optimizer', layout=Layout(font_weight='bold')),
    widgets_dict['optimizer'],
    widgets_dict['learning_rate'],
    widgets_dict['weight_decay'],
    widgets_dict['momentum'],
    widgets_dict['nesterov'],
    widgets_dict['beta1'],
    widgets_dict['beta2'],
    widgets_dict['epsilon'],
])

scheduler_box = VBox([
    Label('Learning Rate Scheduler', layout=Layout(font_weight='bold')),
    widgets_dict['scheduler'],
    widgets_dict['step_size'],
    widgets_dict['gamma'],
    widgets_dict['milestones'],
    widgets_dict['t_max'],
    widgets_dict['eta_min'],
    widgets_dict['warmup_epochs'],
    widgets_dict['plateau_mode'],
    widgets_dict['plateau_factor'],
    widgets_dict['plateau_patience'],
    widgets_dict['max_lr'],
])

loss_box = VBox([
    Label('Loss Function', layout=Layout(font_weight='bold')),
    widgets_dict['loss_type'],
    widgets_dict['focal_alpha'],
    widgets_dict['focal_gamma'],
    widgets_dict['label_smoothing'],
])

advanced_reg_box = VBox([
    Label('Advanced Regularization', layout=Layout(font_weight='bold')),
    widgets_dict['use_mixup'],
    widgets_dict['mixup_alpha'],
    widgets_dict['gradient_clipping'],
    widgets_dict['l1_regularization'],
])

tab3 = VBox([
    HTML("<h3>⚙️ Training Configuration</h3>"),
    HBox([data_box, training_box]),
    HBox([optimizer_box, scheduler_box]),
    HBox([loss_box, advanced_reg_box]),
], layout=Layout(padding='10px'))

print("✓ Tab 3 widgets created")


In [ ]:
# ====================
# TAB 4: EVALUATION & COMPARISON
# ====================

# Model Comparison Mode
widgets_dict['compare_multiple_models'] = Checkbox(
    value=False,
    description='Compare Multiple Models',
    style={'description_width': '180px'},
    layout=Layout(width='500px')
)

widgets_dict['model_list'] = SelectMultiple(
    options=list_models(),
    value=['Baseline_MLP'],
    description='Models to Compare:',
    style={'description_width': '150px'},
    layout=Layout(width='500px', height='120px'),
    disabled=True
)

# Evaluation Metrics
widgets_dict['top_m_values'] = Text(
    value='1,2,4,8,16',
    description='Top-m Values:',
    placeholder='comma-separated',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['compute_baselines'] = Checkbox(
    value=True,
    description='Compute Baselines',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['baselines'] = SelectMultiple(
    options=['Random_1/K', 'Random_M/K', 'Best_Observed', 'Oracle'],
    value=['Random_1/K', 'Random_M/K', 'Best_Observed', 'Oracle'],
    description='Baselines:',
    style={'description_width': '150px'},
    layout=Layout(width='500px', height='100px')
)

# Statistical Analysis
widgets_dict['multi_seed_run'] = Checkbox(
    value=False,
    description='Multi-Seed Run',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['seeds'] = Text(
    value='42,43,44,45,46',
    description='Seeds:',
    placeholder='comma-separated',
    style={'description_width': '150px'},
    layout=Layout(width='500px'),
    disabled=True
)

widgets_dict['confidence_interval'] = FloatSlider(
    value=0.95, min=0.90, max=0.99, step=0.01,
    description='Confidence:',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['statistical_test'] = Dropdown(
    options=['None', 't-test', 'ANOVA', 'Wilcoxon'],
    value='None',
    description='Statistical Test:',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

# Performance Profiling
widgets_dict['track_training_time'] = Checkbox(
    value=True,
    description='Track Training Time',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['track_memory_usage'] = Checkbox(
    value=False,
    description='Track Memory Usage',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['save_checkpoints'] = Checkbox(
    value=True,
    description='Save Checkpoints',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['checkpoint_frequency'] = IntSlider(
    value=5, min=1, max=20,
    description='Checkpoint Freq:',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

# Build Tab 4 layout
comparison_box = VBox([
    Label('Model Comparison', layout=Layout(font_weight='bold')),
    widgets_dict['compare_multiple_models'],
    widgets_dict['model_list'],
])

metrics_box = VBox([
    Label('Evaluation Metrics', layout=Layout(font_weight='bold')),
    widgets_dict['top_m_values'],
    widgets_dict['compute_baselines'],
    widgets_dict['baselines'],
])

statistical_box = VBox([
    Label('Statistical Analysis', layout=Layout(font_weight='bold')),
    widgets_dict['multi_seed_run'],
    widgets_dict['seeds'],
    widgets_dict['confidence_interval'],
    widgets_dict['statistical_test'],
])

profiling_box = VBox([
    Label('Performance Profiling', layout=Layout(font_weight='bold')),
    widgets_dict['track_training_time'],
    widgets_dict['track_memory_usage'],
    widgets_dict['save_checkpoints'],
    widgets_dict['checkpoint_frequency'],
])

tab4 = VBox([
    HTML("<h3>📊 Evaluation & Comparison</h3>"),
    HBox([comparison_box, metrics_box]),
    HBox([statistical_box, profiling_box]),
], layout=Layout(padding='10px'))

# ====================
# TAB 5: VISUALIZATION & EXPORT
# ====================

# Plot Selection
plot_options = [
    'Training Curves', 'Learning Rate Schedule', 'Gradient Flow',
    'CDF', 'PDF/Histogram', 'Box Plot', 'Violin Plot', 'Scatter Plot',
    'Bar Comparison', 'Radar Plot', 'Heatmap Comparison',
    'Probe Phase Heatmap', 'Correlation Matrix', 'Diversity Analysis',
    'Probe Power Distribution', '3D Surface', 'ROC Curves',
    'Precision-Recall', 'Confusion Matrix', 'Top-m Accuracy vs M',
    'Baseline Comparison', 'Eta Distribution'
]

widgets_dict['plot_selection'] = SelectMultiple(
    options=plot_options,
    value=['Training Curves', 'CDF', 'Bar Comparison', 'Probe Phase Heatmap'],
    description='Plot Types:',
    style={'description_width': '150px'},
    layout=Layout(width='500px', height='200px')
)

# Plot Customization
widgets_dict['figure_size'] = Text(
    value='12,6',
    description='Figure Size:',
    placeholder='width,height',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['dpi'] = IntSlider(
    value=150, min=72, max=300,
    description='DPI:',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['plot_style'] = Dropdown(
    options=['seaborn', 'ggplot', 'bmh', 'classic'],
    value='seaborn',
    description='Style:',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['color_palette'] = Dropdown(
    options=['viridis', 'plasma', 'coolwarm', 'Set2', 'tab10'],
    value='viridis',
    description='Color Palette:',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

# Export Options
widgets_dict['save_results'] = Checkbox(
    value=True,
    description='Save Results',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['result_formats'] = SelectMultiple(
    options=['CSV', 'JSON', 'YAML', 'Excel', 'HDF5', 'Pickle'],
    value=['CSV', 'JSON', 'YAML'],
    description='Result Formats:',
    style={'description_width': '150px'},
    layout=Layout(width='500px', height='100px')
)

widgets_dict['save_plots'] = Checkbox(
    value=True,
    description='Save Plots',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['plot_formats'] = SelectMultiple(
    options=['PNG', 'PDF', 'SVG', 'EPS'],
    value=['PNG', 'PDF'],
    description='Plot Formats:',
    style={'description_width': '150px'},
    layout=Layout(width='500px', height='80px')
)

widgets_dict['save_model_weights'] = Checkbox(
    value=True,
    description='Save Model Weights',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['output_directory'] = Text(
    value='results/experiment_{timestamp}',
    description='Output Dir:',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

# Report Generation
widgets_dict['generate_pdf_report'] = Checkbox(
    value=False,
    description='Generate PDF Report',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['report_include'] = SelectMultiple(
    options=['Summary Stats', 'All Plots', 'Config YAML', 'Code Snapshot'],
    value=['Summary Stats', 'All Plots', 'Config YAML'],
    description='Report Include:',
    style={'description_width': '150px'},
    layout=Layout(width='500px', height='100px')
)

# Verbosity
widgets_dict['verbose_output'] = Checkbox(
    value=True,
    description='Verbose Output',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['progress_bars'] = Checkbox(
    value=True,
    description='Progress Bars',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

widgets_dict['log_level'] = Dropdown(
    options=['DEBUG', 'INFO', 'WARNING', 'ERROR'],
    value='INFO',
    description='Log Level:',
    style={'description_width': '150px'},
    layout=Layout(width='500px')
)

# Build Tab 5 layout
plots_box = VBox([
    Label('Plot Selection', layout=Layout(font_weight='bold')),
    widgets_dict['plot_selection'],
])

plot_custom_box = VBox([
    Label('Plot Customization', layout=Layout(font_weight='bold')),
    widgets_dict['figure_size'],
    widgets_dict['dpi'],
    widgets_dict['plot_style'],
    widgets_dict['color_palette'],
])

export_box = VBox([
    Label('Export Options', layout=Layout(font_weight='bold')),
    widgets_dict['save_results'],
    widgets_dict['result_formats'],
    widgets_dict['save_plots'],
    widgets_dict['plot_formats'],
    widgets_dict['save_model_weights'],
    widgets_dict['output_directory'],
])

report_box = VBox([
    Label('Report Generation', layout=Layout(font_weight='bold')),
    widgets_dict['generate_pdf_report'],
    widgets_dict['report_include'],
])

verbosity_box = VBox([
    Label('Verbosity', layout=Layout(font_weight='bold')),
    widgets_dict['verbose_output'],
    widgets_dict['progress_bars'],
    widgets_dict['log_level'],
])

tab5 = VBox([
    HTML("<h3>🎨 Visualization & Export</h3>"),
    HBox([plots_box, plot_custom_box]),
    HBox([export_box, report_box, verbosity_box]),
], layout=Layout(padding='10px'))

# ====================
# ASSEMBLE ALL TABS
# ====================

tabs = Tab(children=[tab1, tab2, tab3, tab4, tab5])
tabs.set_title(0, '🎛️ System & Physics')
tabs.set_title(1, '🧠 Model Architecture')
tabs.set_title(2, '⚙️ Training Config')
tabs.set_title(3, '📊 Evaluation')
tabs.set_title(4, '🎨 Visualization')

print("\n" + "="*70)
print("✅ WIDGET CONTROL PANEL CREATED")
print("="*70)
print(f"Total widgets: {len(widgets_dict)}")
print("Navigate through tabs to configure all parameters")
print("="*70)

# Display the tabs
display(tabs)


In [ ]:
# ============================================================================
# CELL 4: DYNAMIC WIDGET INTERACTIONS & VALIDATION
# ============================================================================

# Storage for additional dynamic widgets
dynamic_widgets = {}

def update_M_max(change):
    """M slider max value = K value (auto-update when K changes)"""
    widgets_dict['M'].max = change['new']
    if widgets_dict['M'].value > change['new']:
        widgets_dict['M'].value = change['new']

def update_phase_bits_enabled(change):
    """Phase Bits only enabled when Phase Mode = Discrete"""
    widgets_dict['phase_bits'].disabled = (change['new'] == 'Continuous')

def update_channel_params_visibility(change):
    """Channel-specific parameters show/hide based on Channel Type"""
    channel_type = change['new']
    
    # Hide all first
    widgets_dict['rician_k'].layout.display = 'none'
    widgets_dict['num_paths'].layout.display = 'none'
    widgets_dict['correlation_distance'].layout.display = 'none'
    widgets_dict['doppler_freq'].layout.display = 'none'
    
    # Show relevant ones
    if channel_type == 'Rician':
        widgets_dict['rician_k'].layout.display = 'flex'
    elif channel_type == 'Sparse':
        widgets_dict['num_paths'].layout.display = 'flex'
    elif channel_type == 'Correlated':
        widgets_dict['correlation_distance'].layout.display = 'flex'
    elif channel_type == 'Time-Varying':
        widgets_dict['doppler_freq'].layout.display = 'flex'

def update_model_architecture_visibility(change):
    """Model architecture widgets change based on Model Type"""
    model_type = change['new']
    
    # Hide all config boxes first
    mlp_config_box.layout.display = 'none'
    cnn_config_box.layout.display = 'none'
    rnn_config_box.layout.display = 'none'
    attention_config_box.layout.display = 'none'
    
    # Show relevant ones
    if model_type in ['MLP', 'ResNet-MLP']:
        mlp_config_box.layout.display = 'flex'
        widgets_dict['use_skip_connections'].disabled = (model_type == 'MLP')
    elif model_type in ['CNN', 'Hybrid-CNN-LSTM']:
        cnn_config_box.layout.display = 'flex'
    
    if model_type in ['LSTM', 'GRU', 'Hybrid-CNN-LSTM']:
        rnn_config_box.layout.display = 'flex'
    
    if model_type in ['Attention', 'Transformer']:
        attention_config_box.layout.display = 'flex'

def update_optimizer_params_visibility(change):
    """Optimizer-specific params show/hide based on Optimizer selection"""
    optimizer = change['new']
    
    # Hide all first
    widgets_dict['momentum'].layout.display = 'none'
    widgets_dict['nesterov'].layout.display = 'none'
    widgets_dict['beta1'].layout.display = 'none'
    widgets_dict['beta2'].layout.display = 'none'
    widgets_dict['epsilon'].layout.display = 'none'
    
    # Show relevant ones
    if optimizer == 'SGD':
        widgets_dict['momentum'].layout.display = 'flex'
        widgets_dict['nesterov'].layout.display = 'flex'
    elif optimizer in ['Adam', 'AdamW', 'Adamax']:
        widgets_dict['beta1'].layout.display = 'flex'
        widgets_dict['beta2'].layout.display = 'flex'
        widgets_dict['epsilon'].layout.display = 'flex'

def update_scheduler_params_visibility(change):
    """Scheduler-specific params show/hide based on Scheduler selection"""
    scheduler = change['new']
    
    # Hide all first
    for widget_name in ['step_size', 'gamma', 'milestones', 't_max', 'eta_min',
                         'warmup_epochs', 'plateau_mode', 'plateau_factor', 
                         'plateau_patience', 'max_lr']:
        widgets_dict[widget_name].layout.display = 'none'
    
    # Show relevant ones
    if scheduler == 'StepLR':
        widgets_dict['step_size'].layout.display = 'flex'
        widgets_dict['gamma'].layout.display = 'flex'
    elif scheduler == 'MultiStepLR':
        widgets_dict['milestones'].layout.display = 'flex'
        widgets_dict['gamma'].layout.display = 'flex'
    elif scheduler == 'CosineAnnealing':
        widgets_dict['t_max'].layout.display = 'flex'
        widgets_dict['eta_min'].layout.display = 'flex'
    elif scheduler == 'CosineWarmup':
        widgets_dict['warmup_epochs'].layout.display = 'flex'
    elif scheduler == 'ReduceLROnPlateau':
        widgets_dict['plateau_mode'].layout.display = 'flex'
        widgets_dict['plateau_factor'].layout.display = 'flex'
        widgets_dict['plateau_patience'].layout.display = 'flex'
    elif scheduler == 'OneCycleLR':
        widgets_dict['max_lr'].layout.display = 'flex'
    elif scheduler == 'ExponentialLR':
        widgets_dict['gamma'].layout.display = 'flex'

def update_loss_params_visibility(change):
    """Loss-specific params show/hide based on Loss selection"""
    loss_type = change['new']
    
    # Hide all first
    widgets_dict['focal_alpha'].layout.display = 'none'
    widgets_dict['focal_gamma'].layout.display = 'none'
    widgets_dict['label_smoothing'].layout.display = 'none'
    
    # Show relevant ones
    if loss_type == 'Focal':
        widgets_dict['focal_alpha'].layout.display = 'flex'
        widgets_dict['focal_gamma'].layout.display = 'flex'
    elif loss_type == 'LabelSmoothing':
        widgets_dict['label_smoothing'].layout.display = 'flex'

def update_mixup_alpha_enabled(change):
    """Mixup Alpha enabled only when Use Mixup is checked"""
    widgets_dict['mixup_alpha'].disabled = not change['new']

def update_multi_seed_enabled(change):
    """Multi-seed options enabled only when Multi-Seed Run is checked"""
    widgets_dict['seeds'].disabled = not change['new']

def update_model_list_enabled(change):
    """Model comparison list enabled only when Compare Multiple Models is checked"""
    widgets_dict['model_list'].disabled = not change['new']

def update_t_max_auto(change):
    """Auto-update T_max when epochs change"""
    widgets_dict['t_max'].value = change['new']

def validate_config():
    """Validate configuration before running"""
    errors = []
    warnings_list = []
    
    # Check M <= K
    if widgets_dict['M'].value > widgets_dict['K'].value:
        errors.append(f"M ({widgets_dict['M'].value}) cannot exceed K ({widgets_dict['K'].value})")
    
    # Check sample sizes are positive
    if widgets_dict['n_train'].value <= 0:
        errors.append("Training samples must be positive")
    if widgets_dict['n_val'].value <= 0:
        errors.append("Validation samples must be positive")
    if widgets_dict['n_test'].value <= 0:
        errors.append("Test samples must be positive")
    
    # Check learning rate is reasonable
    if widgets_dict['learning_rate'].value > 0.1:
        warnings_list.append("Learning rate > 0.1 may be too high")
    
    # Check batch size vs samples
    if widgets_dict['batch_size'].value > widgets_dict['n_train'].value:
        warnings_list.append("Batch size exceeds training samples")
    
    # Check epoch count
    if widgets_dict['epochs'].value < 5:
        warnings_list.append("Less than 5 epochs may not be enough for convergence")
    
    return errors, warnings_list

# ====================
# REGISTER ALL CALLBACKS
# ====================

# System interactions
widgets_dict['K'].observe(update_M_max, names='value')
widgets_dict['phase_mode'].observe(update_phase_bits_enabled, names='value')
widgets_dict['channel_type'].observe(update_channel_params_visibility, names='value')

# Model interactions
widgets_dict['model_type'].observe(update_model_architecture_visibility, names='value')

# Training interactions
widgets_dict['optimizer'].observe(update_optimizer_params_visibility, names='value')
widgets_dict['scheduler'].observe(update_scheduler_params_visibility, names='value')
widgets_dict['loss_type'].observe(update_loss_params_visibility, names='value')
widgets_dict['use_mixup'].observe(update_mixup_alpha_enabled, names='value')
widgets_dict['epochs'].observe(update_t_max_auto, names='value')

# Evaluation interactions
widgets_dict['multi_seed_run'].observe(update_multi_seed_enabled, names='value')
widgets_dict['compare_multiple_models'].observe(update_model_list_enabled, names='value')

# Initialize visibility states
update_phase_bits_enabled({'new': widgets_dict['phase_mode'].value})
update_channel_params_visibility({'new': widgets_dict['channel_type'].value})
update_model_architecture_visibility({'new': widgets_dict['model_type'].value})
update_optimizer_params_visibility({'new': widgets_dict['optimizer'].value})
update_scheduler_params_visibility({'new': widgets_dict['scheduler'].value})
update_loss_params_visibility({'new': widgets_dict['loss_type'].value})

print("\n" + "="*70)
print("✅ DYNAMIC INTERACTIONS CONFIGURED")
print("="*70)
print("Registered interactions:")
print("  1. ✓ M slider max updates with K value")
print("  2. ✓ Phase Bits enabled only when Discrete mode")
print("  3. ✓ Channel-specific parameters auto-show/hide")
print("  4. ✓ Model architecture widgets adapt to model type")
print("  5. ✓ Optimizer parameters adapt to optimizer choice")
print("  6. ✓ Scheduler parameters adapt to scheduler choice")
print("  7. ✓ Loss parameters adapt to loss function choice")
print("  8. ✓ Mixup Alpha enabled with Mixup checkbox")
print("  9. ✓ Multi-seed options enabled with checkbox")
print(" 10. ✓ Model comparison list enabled with checkbox")
print(" 11. ✓ T_max auto-updates with epochs")
print(" 12. ✓ Configuration validation on run")
print("="*70)


In [ ]:
# ============================================================================
# CELL 5: CONFIGURATION BUILDER FROM WIDGETS
# ============================================================================

def build_config_from_widgets():
    """Collect all widget values and build Config object."""
    
    # Parse top-m values
    try:
        top_m_values = [int(x.strip()) for x in widgets_dict['top_m_values'].value.split(',')]
    except:
        top_m_values = [1, 2, 4, 8]
    
    # Parse MLP hidden sizes
    hidden_sizes = [
        widgets_dict['mlp_layer_1'].value,
        widgets_dict['mlp_layer_2'].value,
        widgets_dict['mlp_layer_3'].value,
    ][:widgets_dict['mlp_num_layers'].value]
    
    # Build config
    config = Config(
        system=SystemConfig(
            N=widgets_dict['N'].value,
            K=widgets_dict['K'].value,
            M=widgets_dict['M'].value,
            P_tx=widgets_dict['P_tx'].value,
            sigma_h_sq=widgets_dict['sigma_h_sq'].value,
            sigma_g_sq=widgets_dict['sigma_g_sq'].value,
            phase_mode=widgets_dict['phase_mode'].value.lower(),
            phase_bits=widgets_dict['phase_bits'].value,
            probe_type=list(widgets_dict['probe_methods'].value)[0] if widgets_dict['probe_methods'].value else 'continuous',
        ),
        data=DataConfig(
            n_train=widgets_dict['n_train'].value,
            n_val=widgets_dict['n_val'].value,
            n_test=widgets_dict['n_test'].value,
            seed=widgets_dict['seed'].value,
            normalize_input=widgets_dict['normalize_input'].value,
            normalization_type=widgets_dict['normalization_type'].value,
        ),
        model=ModelConfig(
            hidden_sizes=hidden_sizes,
            dropout_prob=widgets_dict['dropout'].value,
            use_batch_norm=widgets_dict['batch_normalization'].value,
        ),
        training=TrainingConfig(
            batch_size=widgets_dict['batch_size'].value,
            learning_rate=widgets_dict['learning_rate'].value,
            weight_decay=widgets_dict['weight_decay'].value,
            n_epochs=widgets_dict['epochs'].value,
            early_stop_patience=widgets_dict['early_stop_patience'].value,
            eval_interval=widgets_dict['validation_interval'].value,
            device=device,
        ),
        eval=EvalConfig(
            top_m_values=top_m_values,
        )
    )
    
    return config

def get_experiment_metadata():
    """Collect metadata about the experiment."""
    metadata = {
        'timestamp': datetime.now().strftime('%Y-%m-%d_%H-%M-%S'),
        'probe_methods': list(widgets_dict['probe_methods'].value),
        'model_type': widgets_dict['model_type'].value,
        'optimizer': widgets_dict['optimizer'].value,
        'scheduler': widgets_dict['scheduler'].value,
        'loss_type': widgets_dict['loss_type'].value,
        'compare_models': widgets_dict['compare_multiple_models'].value,
        'multi_seed': widgets_dict['multi_seed_run'].value,
        'device': str(device),
        'pytorch_version': torch.__version__,
        'numpy_version': np.__version__,
    }
    return metadata

print("✅ Configuration builder ready")


In [ ]:
# ============================================================================
# CELL 6: EXPERIMENT RUNNER
# ============================================================================

# Global state
experiment_running = False
experiment_results = {}
experiment_history = {}

# Output area for progress
progress_output = Output()

def run_single_experiment(config, probe_type, verbose=True):
    """Run a single experiment with given configuration."""
    
    with progress_output:
        if verbose:
            print(f"\n{'='*70}")
            print(f"Running experiment with probe_type: {probe_type}")
            print(f"{'='*70}")
        
        # Generate probe bank
        probe_bank = get_probe_bank(
            N=config.system.N,
            K=config.system.K,
            probe_type=probe_type,
            phase_mode=config.system.phase_mode,
            phase_bits=config.system.phase_bits,
            seed=config.data.seed
        )
        
        # Create dataloaders
        train_loader, val_loader, test_loader = create_dataloaders(
            probe_bank=probe_bank,
            config=config
        )
        
        # Create model
        model = create_model(config).to(config.training.device)
        if verbose:
            n_params = count_parameters(model)
            print(f"✓ Model created: {n_params:,} parameters")
        
        # Train model
        start_time = time.time()
        trained_model, history = train(model, train_loader, val_loader, config)
        training_time = time.time() - start_time
        
        if verbose:
            print(f"✓ Training completed in {training_time:.1f}s")
        
        # Evaluate
        results = evaluate_model(trained_model, test_loader, config)
        results['training_time'] = training_time
        results['n_parameters'] = n_params
        
        if verbose:
            print(f"✓ Evaluation completed")
            print(f"  Best eta: {results['eta_best']:.4f}")
            print(f"  Top-1 accuracy: {results['top1_accuracy']:.2%}")
        
        return {
            'model': trained_model,
            'results': results,
            'history': history,
            'probe_bank': probe_bank,
        }

def run_experiment_click(b):
    """Main experiment execution when button is clicked."""
    global experiment_running, experiment_results, experiment_history
    
    # Prevent double-clicks
    if experiment_running:
        with progress_output:
            print("⚠️  Experiment already running!")
        return
    
    # Clear previous output
    progress_output.clear_output()
    
    # Disable button
    experiment_running = True
    run_button.disabled = True
    run_button.description = "⏳ Running..."
    
    try:
        # Validate configuration
        with progress_output:
            print("\n" + "="*70)
            print("🚀 STARTING EXPERIMENT")
            print("="*70)
            
            errors, warnings_list = validate_config()
            
            if errors:
                print("\n❌ Configuration Errors:")
                for error in errors:
                    print(f"  • {error}")
                raise ValueError("Configuration validation failed")
            
            if warnings_list:
                print("\n⚠️  Configuration Warnings:")
                for warning in warnings_list:
                    print(f"  • {warning}")
                print()
        
        # Build config
        config = build_config_from_widgets()
        metadata = get_experiment_metadata()
        
        with progress_output:
            config.print_config()
        
        # Create output directory
        output_dir = widgets_dict['output_directory'].value.replace(
            '{timestamp}', metadata['timestamp']
        )
        output_path = Path(output_dir)
        output_path.mkdir(parents=True, exist_ok=True)
        
        # Save config
        if widgets_dict['save_results'].value:
            config_path = output_path / 'config.yaml'
            with open(config_path, 'w') as f:
                yaml.dump({
                    'system': config.system.__dict__,
                    'data': config.data.__dict__,
                    'model': config.model.__dict__,
                    'training': config.training.__dict__,
                    'eval': config.eval.__dict__,
                    'metadata': metadata,
                }, f)
            with progress_output:
                print(f"\n✓ Config saved to {config_path}")
        
        # Run experiments
        experiment_results = {}
        experiment_history = {}
        
        probe_methods = list(widgets_dict['probe_methods'].value)
        
        if widgets_dict['multi_seed_run'].value:
            seeds = [int(x.strip()) for x in widgets_dict['seeds'].value.split(',')]
        else:
            seeds = [config.data.seed]
        
        total_runs = len(probe_methods) * len(seeds)
        run_count = 0
        
        with progress_output:
            print(f"\n📊 Running {total_runs} experiment(s)...")
            print(f"  Probe methods: {probe_methods}")
            print(f"  Seeds: {seeds}")
            print()
        
        for probe_type in probe_methods:
            experiment_results[probe_type] = {}
            experiment_history[probe_type] = {}
            
            for seed in seeds:
                run_count += 1
                
                # Update config seed
                config.data.seed = seed
                
                with progress_output:
                    print(f"\n[{run_count}/{total_runs}] {probe_type} (seed={seed})")
                
                result = run_single_experiment(config, probe_type, verbose=True)
                
                experiment_results[probe_type][seed] = result['results']
                experiment_history[probe_type][seed] = result['history']
        
        # Success message
        with progress_output:
            print("\n" + "="*70)
            print("✅ EXPERIMENT COMPLETED SUCCESSFULLY")
            print("="*70)
            print(f"Results saved to: {output_path}")
            print("\nScroll down to view results dashboard →")
            print("="*70)
    
    except Exception as e:
        with progress_output:
            print(f"\n❌ ERROR: {str(e)}")
            import traceback
            traceback.print_exc()
    
    finally:
        # Re-enable button
        experiment_running = False
        run_button.disabled = False
        run_button.description = "🚀 RUN EXPERIMENT"

# Create run button
run_button = Button(
    description="🚀 RUN EXPERIMENT",
    button_style='success',
    layout=Layout(width='300px', height='60px'),
    style={'font_weight': 'bold', 'font_size': '16px'}
)
run_button.on_click(run_experiment_click)

print("\n" + "="*70)
print("✅ EXPERIMENT RUNNER READY")
print("="*70)
print("Click the button below to start!")
print("="*70)

display(run_button)


In [ ]:
# ============================================================================
# CELL 7: REAL-TIME PROGRESS DISPLAY
# ============================================================================

# Display progress output
display(HTML("<h3>📊 Experiment Progress</h3>"))
display(progress_output)

print("✅ Progress display ready")
print("Progress will be shown here during experiment execution...")


In [ ]:
# ============================================================================
# CELL 8: RESULTS VISUALIZATION DASHBOARD
# ============================================================================

def display_results_dashboard():
    """Display comprehensive results dashboard."""
    
    if not experiment_results:
        print("⚠️  No results yet. Run an experiment first!")
        return
    
    print("\n" + "="*70)
    print("📊 RESULTS DASHBOARD")
    print("="*70)
    
    # Summary Statistics Table
    print("\n### Summary Statistics")
    print("-" * 70)
    
    summary_data = []
    for probe_type, results_dict in experiment_results.items():
        for seed, results in results_dict.items():
            summary_data.append({
                'Probe Type': probe_type,
                'Seed': seed,
                'Best Eta': f"{results['eta_best']:.4f}",
                'Top-1 Acc': f"{results['top1_accuracy']:.2%}",
                'Training Time': f"{results.get('training_time', 0):.1f}s",
                'Parameters': f"{results.get('n_parameters', 0):,}",
            })
    
    df_summary = pd.DataFrame(summary_data)
    display(df_summary)
    
    # Best Model Identification
    print("\n### 🏆 Best Performing Configuration")
    print("-" * 70)
    
    best_eta = 0
    best_probe = None
    best_seed = None
    
    for probe_type, results_dict in experiment_results.items():
        for seed, results in results_dict.items():
            if results['eta_best'] > best_eta:
                best_eta = results['eta_best']
                best_probe = probe_type
                best_seed = seed
    
    print(f"Probe Type: {best_probe}")
    print(f"Seed: {best_seed}")
    print(f"Best Eta: {best_eta:.4f}")
    print("-" * 70)
    
    # Generate selected plots
    print("\n### �� Visualizations")
    print("-" * 70)
    
    selected_plots = list(widgets_dict['plot_selection'].value)
    fig_width, fig_height = [float(x) for x in widgets_dict['figure_size'].value.split(',')]
    
    for plot_name in selected_plots:
        print(f"\nGenerating: {plot_name}...")
        
        try:
            # Map plot names to functions
            if plot_name == 'Training Curves' and experiment_history:
                fig, axes = plt.subplots(1, 2, figsize=(fig_width, fig_height/2))
                
                for probe_type, history_dict in experiment_history.items():
                    for seed, history in history_dict.items():
                        label = f"{probe_type} (seed={seed})" if len(history_dict) > 1 else probe_type
                        axes[0].plot(history['train_loss'], label=label, alpha=0.7)
                        axes[1].plot(history['val_eta'], label=label, alpha=0.7)
                
                axes[0].set_xlabel('Epoch')
                axes[0].set_ylabel('Training Loss')
                axes[0].set_title('Training Loss Curves')
                axes[0].legend()
                axes[0].grid(True, alpha=0.3)
                
                axes[1].set_xlabel('Epoch')
                axes[1].set_ylabel('Validation Eta')
                axes[1].set_title('Validation Performance')
                axes[1].legend()
                axes[1].grid(True, alpha=0.3)
                
                plt.tight_layout()
                plt.show()
            
            elif plot_name == 'CDF':
                plt.figure(figsize=(fig_width/2, fig_height/2))
                
                for probe_type, results_dict in experiment_results.items():
                    for seed, results in results_dict.items():
                        eta_vals = results.get('eta_all', [])
                        if len(eta_vals) > 0:
                            sorted_eta = np.sort(eta_vals)
                            cdf = np.arange(1, len(sorted_eta) + 1) / len(sorted_eta)
                            label = f"{probe_type} (seed={seed})" if len(results_dict) > 1 else probe_type
                            plt.plot(sorted_eta, cdf, label=label, linewidth=2)
                
                plt.xlabel('Eta (η)')
                plt.ylabel('CDF')
                plt.title('Cumulative Distribution of Performance')
                plt.legend()
                plt.grid(True, alpha=0.3)
                plt.tight_layout()
                plt.show()
            
            elif plot_name == 'Bar Comparison':
                plt.figure(figsize=(fig_width/2, fig_height/2))
                
                probe_types = []
                eta_means = []
                
                for probe_type, results_dict in experiment_results.items():
                    etas = [r['eta_best'] for r in results_dict.values()]
                    probe_types.append(probe_type)
                    eta_means.append(np.mean(etas))
                
                plt.bar(probe_types, eta_means, color=sns.color_palette('viridis', len(probe_types)))
                plt.xlabel('Probe Type')
                plt.ylabel('Mean Best Eta')
                plt.title('Performance Comparison Across Probe Types')
                plt.xticks(rotation=45, ha='right')
                plt.grid(True, alpha=0.3, axis='y')
                plt.tight_layout()
                plt.show()
            
            elif plot_name == 'Eta Distribution':
                n_probes = len(experiment_results)
                fig, axes = plt.subplots(1, n_probes, figsize=(fig_width, fig_height/2))
                if n_probes == 1:
                    axes = [axes]
                
                for idx, (probe_type, results_dict) in enumerate(experiment_results.items()):
                    all_etas = []
                    for results in results_dict.values():
                        all_etas.extend(results.get('eta_all', []))
                    
                    if len(all_etas) > 0:
                        axes[idx].hist(all_etas, bins=50, alpha=0.7, edgecolor='black')
                        axes[idx].set_xlabel('Eta (η)')
                        axes[idx].set_ylabel('Frequency')
                        axes[idx].set_title(f'{probe_type}')
                        axes[idx].grid(True, alpha=0.3)
                
                plt.tight_layout()
                plt.show()
            
        except Exception as e:
            print(f"  ⚠️  Error generating {plot_name}: {str(e)}")
    
    print("\n" + "="*70)
    print("✅ Results dashboard complete!")
    print("="*70)

# Button to display results
results_button = Button(
    description="📊 Show Results Dashboard",
    button_style='info',
    layout=Layout(width='300px', height='50px')
)
results_button.on_click(lambda b: display_results_dashboard())

display(HTML("<h3>📊 Results Visualization</h3>"))
display(results_button)


In [ ]:
# ============================================================================
# CELL 9: EXPORT & SAVE FUNCTIONS
# ============================================================================

def export_results():
    """Export results in selected formats."""
    
    if not experiment_results:
        print("⚠️  No results to export!")
        return
    
    # Get output directory
    metadata = get_experiment_metadata()
    output_dir = widgets_dict['output_directory'].value.replace(
        '{timestamp}', metadata['timestamp']
    )
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)
    
    print(f"\n📁 Exporting results to: {output_path}")
    print("-" * 70)
    
    # Prepare data for export
    export_data = []
    for probe_type, results_dict in experiment_results.items():
        for seed, results in results_dict.items():
            row = {
                'probe_type': probe_type,
                'seed': seed,
                'eta_best': results['eta_best'],
                'top1_accuracy': results['top1_accuracy'],
                'training_time': results.get('training_time', 0),
                'n_parameters': results.get('n_parameters', 0),
            }
            # Add top-m accuracies
            for m, acc in results.get('top_m_accuracies', {}).items():
                row[f'top{m}_accuracy'] = acc
            export_data.append(row)
    
    df = pd.DataFrame(export_data)
    
    # Export in selected formats
    result_formats = list(widgets_dict['result_formats'].value)
    
    if 'CSV' in result_formats:
        csv_path = output_path / 'results.csv'
        df.to_csv(csv_path, index=False)
        print(f"✓ CSV: {csv_path}")
    
    if 'JSON' in result_formats:
        json_path = output_path / 'results.json'
        with open(json_path, 'w') as f:
            json.dump({
                'results': export_data,
                'metadata': metadata,
            }, f, indent=2)
        print(f"✓ JSON: {json_path}")
    
    if 'YAML' in result_formats:
        yaml_path = output_path / 'results.yaml'
        with open(yaml_path, 'w') as f:
            yaml.dump({
                'results': export_data,
                'metadata': metadata,
            }, f)
        print(f"✓ YAML: {yaml_path}")
    
    if 'Excel' in result_formats:
        try:
            excel_path = output_path / 'results.xlsx'
            df.to_excel(excel_path, index=False)
            print(f"✓ Excel: {excel_path}")
        except ImportError:
            print(f"⚠️  Excel export requires openpyxl: pip install openpyxl")
    
    if 'Pickle' in result_formats:
        pickle_path = output_path / 'results.pkl'
        with open(pickle_path, 'wb') as f:
            pickle.dump({
                'results': experiment_results,
                'history': experiment_history,
                'metadata': metadata,
            }, f)
        print(f"✓ Pickle: {pickle_path}")
    
    print("-" * 70)
    print("✅ Export complete!")

# Export button
export_button = Button(
    description="💾 Export Results",
    button_style='warning',
    layout=Layout(width='300px', height='50px')
)
export_button.on_click(lambda b: export_results())

display(HTML("<h3>💾 Export Results</h3>"))
display(export_button)

print("✅ Export functions ready")


In [ ]:
# ============================================================================
# CELL 10: HELPER FUNCTIONS & UTILITIES
# ============================================================================

def load_config_from_yaml(yaml_path):
    """Load configuration from YAML file and apply to widgets."""
    with open(yaml_path, 'r') as f:
        config_dict = yaml.safe_load(f)
    
    # Apply system config
    if 'system' in config_dict:
        for key, value in config_dict['system'].items():
            if key in widgets_dict:
                widgets_dict[key].value = value
    
    # Apply data config
    if 'data' in config_dict:
        for key, value in config_dict['data'].items():
            widget_key = key if key in widgets_dict else f'n_{key}' if f'n_{key}' in widgets_dict else None
            if widget_key and widget_key in widgets_dict:
                widgets_dict[widget_key].value = value
    
    # Apply model config
    if 'model' in config_dict:
        if 'hidden_sizes' in config_dict['model']:
            hidden = config_dict['model']['hidden_sizes']
            widgets_dict['mlp_num_layers'].value = len(hidden)
            for i, size in enumerate(hidden[:3], 1):
                widgets_dict[f'mlp_layer_{i}'].value = size
    
    # Apply training config
    if 'training' in config_dict:
        for key, value in config_dict['training'].items():
            if key == 'n_epochs':
                widgets_dict['epochs'].value = value
            elif key in widgets_dict:
                widgets_dict[key].value = value
    
    print(f"✅ Configuration loaded from {yaml_path}")

def reset_to_defaults():
    """Reset all widgets to default values."""
    # System defaults
    widgets_dict['N'].value = 32
    widgets_dict['K'].value = 64
    widgets_dict['M'].value = 8
    widgets_dict['phase_mode'].value = 'Continuous'
    widgets_dict['probe_methods'].value = ['sobol', 'hadamard']
    
    # Model defaults
    widgets_dict['model_type'].value = 'MLP'
    widgets_dict['mlp_num_layers'].value = 3
    widgets_dict['mlp_layer_1'].value = 512
    widgets_dict['mlp_layer_2'].value = 256
    widgets_dict['mlp_layer_3'].value = 128
    widgets_dict['dropout'].value = 0.1
    widgets_dict['batch_normalization'].value = True
    
    # Training defaults
    widgets_dict['n_train'].value = 50000
    widgets_dict['n_val'].value = 5000
    widgets_dict['n_test'].value = 5000
    widgets_dict['epochs'].value = 50
    widgets_dict['batch_size'].value = 128
    widgets_dict['learning_rate'].value = 1e-3
    widgets_dict['optimizer'].value = 'Adam'
    widgets_dict['scheduler'].value = 'None'
    
    print("✅ All widgets reset to defaults")

def load_preset(preset_name):
    """Load predefined configuration presets."""
    presets = {
        'quick_test': {
            'N': 16, 'K': 32, 'M': 4,
            'n_train': 5000, 'n_val': 500, 'n_test': 500,
            'epochs': 10, 'batch_size': 64,
        },
        'full_training': {
            'N': 32, 'K': 64, 'M': 8,
            'n_train': 50000, 'n_val': 5000, 'n_test': 5000,
            'epochs': 50, 'batch_size': 128,
        },
        'paper_reproduction': {
            'N': 32, 'K': 64, 'M': 8,
            'n_train': 100000, 'n_val': 10000, 'n_test': 10000,
            'epochs': 100, 'batch_size': 256,
            'multi_seed_run': True,
            'seeds': '42,43,44,45,46,47,48,49,50,51',
        },
    }
    
    if preset_name not in presets:
        print(f"❌ Unknown preset: {preset_name}")
        print(f"Available presets: {list(presets.keys())}")
        return
    
    preset = presets[preset_name]
    
    for key, value in preset.items():
        if key in widgets_dict:
            widgets_dict[key].value = value
    
    print(f"✅ Loaded preset: {preset_name}")

def estimate_training_time(config=None):
    """Estimate training time based on configuration."""
    if config is None:
        config = build_config_from_widgets()
    
    # Rough estimation based on dataset size and model size
    samples_per_epoch = config.data.n_train / config.training.batch_size
    hidden_size_factor = sum(config.model.hidden_sizes) / 1000
    
    # Baseline: ~0.5 seconds per epoch for default config
    time_per_epoch = 0.5 * hidden_size_factor * (samples_per_epoch / 390)
    total_time = time_per_epoch * config.training.n_epochs
    
    print(f"📊 Training Time Estimate:")
    print(f"  Time per epoch: ~{time_per_epoch:.1f}s")
    print(f"  Total time: ~{total_time:.1f}s ({total_time/60:.1f} minutes)")
    print(f"  (Actual time may vary based on hardware)")

def get_system_info():
    """Display system information."""
    print("💻 System Information:")
    print(f"  Device: {device}")
    if torch.cuda.is_available():
        print(f"  GPU: {torch.cuda.get_device_name(0)}")
        print(f"  GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
        print(f"  CUDA Version: {torch.version.cuda}")
    print(f"  PyTorch: {torch.__version__}")
    print(f"  NumPy: {np.__version__}")
    print(f"  Python: {sys.version.split()[0]}")

# Utility buttons
utility_box = VBox([
    HTML("<h3>🛠️ Utility Functions</h3>"),
    Button(description="Reset to Defaults", button_style='', 
           layout=Layout(width='200px')),
    Button(description="Quick Test Preset", button_style='', 
           layout=Layout(width='200px')),
    Button(description="Full Training Preset", button_style='', 
           layout=Layout(width='200px')),
    Button(description="Estimate Training Time", button_style='', 
           layout=Layout(width='200px')),
    Button(description="System Info", button_style='', 
           layout=Layout(width='200px')),
])

# Wire up button callbacks
utility_box.children[1].on_click(lambda b: reset_to_defaults())
utility_box.children[2].on_click(lambda b: load_preset('quick_test'))
utility_box.children[3].on_click(lambda b: load_preset('full_training'))
utility_box.children[4].on_click(lambda b: estimate_training_time())
utility_box.children[5].on_click(lambda b: get_system_info())

display(utility_box)

print("\n" + "="*70)
print("✅ ALL CELLS COMPLETE - PhD RESEARCH DASHBOARD READY!")
print("="*70)
print("\n📖 Usage Instructions:")
print("  1. Configure parameters in the tabs above")
print("  2. Click 'RUN EXPERIMENT' to start")
print("  3. Monitor progress in real-time")
print("  4. View results dashboard")
print("  5. Export results in your preferred format")
print("\n🎓 Happy researching!")
print("="*70)
